## State of Washington - Licensee - Rolling Farms

* UBI: 603350787

We'll be using the [`cannapy`](https://github.com/CannabisData/cannapy) library to access the portal data.  `cannapy` aims to provide an abstract interface for accessing and working with *Cannabis* data from around the world.  It utilizes [xmunoz](https://github.com/xmunoz)'s [`sodapy`](https://github.com/xmunoz/sodapy) client to access Socrata-based open data portals and can return data loaded into [Pandas DataFrames](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

### Dataset: Licensed Businesses

* Canonical Dataset ID: **bhbp-x4eb**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/Licensing/Licensed-Businesses/u3zh-ri66
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/bhbp-x4eb

In [1]:
import time
import cannapy.us.wa.wslcb.portal as wslcb
import pandas as pd

In [2]:
# Specify your own Socrata App Token if you plan to experiment
app_token = 'XaB9MBqc81C3KT4Vps6Wh5LZt'

# Instantiate a cannapy interface to the WSLCB open data portal
portal = wslcb.WSLCBPortal(app_token)

# We'll be using the Licensed Businesses dataset
dataset_id = 'bhbp-x4eb'

# And we're looking for data on a particular licensee
licensee_ubi = '603352830'

In [3]:
# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Wed Jun 27 14:02:12 2018


In [4]:
# Retrieve the dataset preloaded into a Pandas DataFrame
licenses = portal.get_dataframe(dataset_id)

In [5]:
# The UBI column uniquely identifies each licensee, but obscures ownership of multiple licenses by the same entity.
# Let's break that column apart into its constituent parts:
# Unified Business Identifier (UBI): first nine digits
# Business ID Number: next three digits
# Location Number: last four digits
df_v2 = licenses.rename(columns={'ubi': 'ubi_source'})
df_v2['ubi'] = df_v2.ubi_source.str[0:9]
df_v2['ubi_business_id'] = df_v2.ubi_source.str[9:12]
df_v2['ubi_location'] = df_v2.ubi_source.str[12:]
licensee_licenses = df_v2.loc[df_v2['ubi'] == licensee_ubi]
licensee_licenses

,license,type,createdate,active,organization,address,address_line_2,city,state,zip,county,dayphone,ubi_source,ubi,ubi_business_id,ubi_location
1600,416113,MARIJUANA PRODUCER TIER 3/MARIJUANA PROCESSOR,20180619,PENDING (ISSUED),ROLLING FARMS,18520 67TH AVE NE,NaN,ARLINGTON,WA,982230000,SNOHOMISH,4258029871,6033528300010000,603352830,001,0000


### Dataset: Enforcement Visits

* Canonical Dataset ID: **w7wg-8m52**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/dataset/Enforcement-Visits-Dataset/jizx-thwg
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/w7wg-8m52

In [6]:
# Let's see how many enforcement visits the licensee has hosted
dataset_id = 'w7wg-8m52'

# Select the licensee's license number
# TODO: find a way to do this without hardcoding the row number
licensee_license_number = licensee_licenses.loc[1600, 'license']

# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Mon Feb 25 13:34:36 2019


In [7]:
# Retrieve the dataset preloaded into a Pandas DataFrame
enforcement_visits = portal.get_dataframe(dataset_id)

# Suppress the chained assignment warning: https://stackoverflow.com/a/20627316/7622699
pd.options.mode.chained_assignment = None

# Pull aside the enforcement visits by the selected licensee
licensee_enforcement_visits = enforcement_visits.loc[enforcement_visits['license_number'] == licensee_license_number]

# Sort the DataFrame by 'date'
licensee_enforcement_visits.sort_values(by='date', inplace=True)

licensee_enforcement_visits

,date,license_number,city_name,county_name,activity
11358,2017-06-13T00:00:00.000,416113,ARLINGTON,SNOHOMISH,Marijuana Premises Check
8659,2017-10-30T00:00:00.000,416113,ARLINGTON,SNOHOMISH,Marijuana Premises Check
6564,2018-02-13T00:00:00.000,416113,ARLINGTON,SNOHOMISH,Marijuana Premises Check
3996,2018-07-06T00:00:00.000,416113,ARLINGTON,SNOHOMISH,Marijuana Premises Check
1556,2018-11-08T00:00:00.000,416113,ARLINGTON,SNOHOMISH,Marijuana Premises Check
1478,2018-11-13T00:00:00.000,416113,ARLINGTON,SNOHOMISH,Marijuana Premises Check
652,2018-12-21T00:00:00.000,416113,ARLINGTON,SNOHOMISH,Marijuana Premises Check


### Dataset: Violations

* Canonical Dataset ID: **dgm4-3cm6**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/dataset/Violations-Dataset/dx3i-tzh2
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/dgm4-3cm6

In [8]:
# Let's pull up all of the licensee's violations
dataset_id = 'dgm4-3cm6'

# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Mon Feb 25 13:33:58 2019


In [9]:
# Retrieve the dataset preloaded into a Pandas DataFrame
violations = portal.get_dataframe(dataset_id)

# Pull aside the violations by the selected licensee
licensee_violations = violations.loc[violations['license_number'] == licensee_license_number]

# Sort the DataFrame by 'visit_date'
licensee_violations.sort_values(by='visit_date', inplace=True)

licensee_violations

,visit_date,license_number,county_name,city_name,case,violation_code,wac_code,penalty_type
877,2017-10-30T00:00:00.000,416113,SNOHOMISH,ARLINGTON,4U7303A,NaN,314.55.155,AVN
878,2017-10-30T00:00:00.000,416113,SNOHOMISH,ARLINGTON,4U7303A,NaN,314.55.083(4),AVN
486,2018-06-07T00:00:00.000,416113,SNOHOMISH,ARLINGTON,7G8158C,NaN,314.55.020,AVN
489,2018-06-07T00:00:00.000,416113,SNOHOMISH,ARLINGTON,7G8158C,NaN,314.55.104,AVN
80,2019-01-02T00:00:00.000,416113,SNOHOMISH,ARLINGTON,7F9002A,NaN,314.55.020,AVN
82,2019-01-02T00:00:00.000,416113,SNOHOMISH,ARLINGTON,7F9002A,NaN,314.55.083(4),AVN
83,2019-01-02T00:00:00.000,416113,SNOHOMISH,ARLINGTON,7F9002A,NaN,314.55.097,AVN
86,2019-01-02T00:00:00.000,416113,SNOHOMISH,ARLINGTON,7F9002A,NaN,314.55.083(3),Written Warning
